In [ ]:
%matplotlib inline

In [ ]:
import os
DATADIR = os.path.join(os.getcwd(),"..", "ClassPrep")
print(os.path.exists(DATADIR))
import csv
from collections import defaultdict
import gzip
import pickle
import networkx as nx
from IPython.display import Image, clear_output
import warnings
warnings.simplefilter('ignore')
import nxdrawing as nxd

### Create a MultiGraph of 'To'/'From' Relationships

In [ ]:
import re
re_me = re.compile(r"""Brian\.Chapman@utah\.edu""",re.I)

### [``MultiGraph``](https://networkx.github.io/documentation/latest/reference/classes.multigraph.html)

We're going to put the e-mail information into a NetworkX ``MultiGraph``. The ``MultiGraph`` allows us to have multiple edges between nodes.

In this graph nodes will be senders and receivers and edges will be particular messages.

In [ ]:
my_email = nx.MultiDiGraph()

with open(os.path.join(DATADIR,
                            "my_emails_2017.txt"),'rt') as f:
    reader = csv.reader( f,delimiter="\t" )

    for row in reader:
        try:
            my_email.add_edge(row[0].lower(),row[1].lower())
        except Exception as error:
            print(error)
            pass


## How large is this graph?


In [ ]:
my_email.number_of_nodes(), my_email.number_of_edges()

## Challenge: To whom did you send the most e-mails?

#### Hint: `neighbors` or `successors`

## Challenge: From whom did you receive the most e-mails?

#### Hint: `predecessors`

### Connected Subgraphs

Graphs can consist of distinct components that are disconnected from each other [connected_component_subgraphs](https://networkx.github.io/documentation/latest/reference/generated/networkx.algorithms.components.connected.connected_component_subgraphs.html?highlight=connected_component_subgraphs) creates distinct graphs for each connected component and returns them in a list. This is only defined for undirected graphs.

We also use the [``sort``](https://docs.python.org/3/library/stdtypes.html#list.sort) method of the list to sort the subgraphs by the number of nodes in each graph.

>*key* specifies a function of one argument that is used to extract a comparison key from each list element (for example, key=str.lower). The key corresponding to each item in the list is calculated once and then used for the entire sorting process. The default value of None means that list items are sorted directly without calculating a separate key value.

* We use list comprehension to keep the subgraphs that have more than two nodes.
* We use an [anonymous lambda function](https://docs.python.org/3/howto/functional.html#small-functions-and-the-lambda-expression) to do the sorting.

In [ ]:
subgraphs = [g for g in nx.connected_component_subgraphs(my_email.to_undirected()) if g.number_of_nodes() > 2]
subgraphs.sort(key=lambda g: g.number_of_nodes())
print("The number of subgraphs is %d"%len(subgraphs))
print([g.number_of_nodes() for g in subgraphs])

In [ ]:
for n in sg.nodes():
    print(n)

### With whom are my most frequent e-mail exchanges?

In [ ]:
main_email = subgraphs[-1]
edges = main_email.edges()
edge_count2 = {}
mail_count_limit = 35
for n in main_email.nodes():
    neighbors = main_email.neighbors(n)
    for nn in neighbors:
        if "Brian.Chapman@utah.edu" in n or "Brian.Chapman@utah.edu" in nn:
            key = [n,nn]
            key.sort()
            edge_count2[tuple(key)] = main_email.number_of_edges(n,nn)

    

In [ ]:
ec = list(edge_count2.items())
ec.sort(key=lambda x:x[1], reverse=True)

In [ ]:
print("%s   %s\t %s"%("Node1".ljust(40),"Node2".rjust(40), "count".ljust(10)))
print()
for e,c in ec[:100]:
    print("%s \u21E8 %s\t% 3d"%(e[0].ljust(40), e[1].rjust(40),c))

## Challenge: Select a random subset of the graph to get a better visualization

#### Hints:

1. Use random.shuffle and slicing
1. Use nx.subgraph

## Challenge: Convert the subgraph to a Graph and Redraw the subgraph